### Import libraries

In [ ]:
import os, sys
print("Current working dir : %s" % os.getcwd())

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import joblib # save/load models
import pickle # save/load features
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score, roc_curve, auc, classification_report, confusion_matrix
from sklearn import preprocessing 
from sklearn.metrics import confusion_matrix # for confusion_matrix
from sklearn.utils.multiclass import unique_labels# for confusion_matrix

import warnings
warnings.filterwarnings('ignore')

### Functions used for experiments

In [ ]:
def load_model(filename_model):
    return joblib.load(filename_model)

In [ ]:
def load_objs(filename_objs):
    with open(filename_objs, 'rb') as f:
        try:
            loaded_objs = pickle.load(f)
            training_features = loaded_objs[0]
            test_features = loaded_objs[1]
            training_target  = loaded_objs[2]
            test_target  = loaded_objs[3]
            X_train = loaded_objs[4]
            X_val = loaded_objs[5]
            y_train = loaded_objs[6]
            y_val = loaded_objs[7]
            class_weight  = loaded_objs[8],
            class_names  = loaded_objs[9]
            n_classes = loaded_objs[10]
        except:
            print("Something went wrong when writing to the file")

    return (training_features, test_features, training_target, 
            test_target,X_train, X_val, y_train, y_val,
            class_weight,class_names,n_classes)

In [ ]:
def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    from sklearn import preprocessing
    lb = preprocessing.LabelBinarizer()
    preprocessing.LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)
    return roc_auc_score(y_test, y_pred, average=average)

In [ ]:
def evaluate_predictions(y_test, predictions):
    accuracy = accuracy_score(y_test, predictions)

    balanced_accuracy = balanced_accuracy_score(y_test, predictions)
    
    f1_score_weighted = f1_score(y_test, predictions, average='weighted')

    roc_auc_score = multiclass_roc_auc_score(y_test, predictions)
    out = "%.2f" % (roc_auc_score * 100.0)

    return out

In [ ]:
def rocauc_perclass(y_pred, y_test, class_names, n_classes):
    # binarize to see roc_auc_score per class
    y_pred_binarize = preprocessing.label_binarize(y_pred, classes=class_names)
    y_test_binarize = preprocessing.label_binarize(y_test, classes=class_names)
    
    # compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    roc_auc_all = []
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test_binarize[:, i], y_pred_binarize[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # compute micro-average ROC curve and ROC area
    fpr["macro"], tpr["macro"], _ = roc_curve(y_test_binarize.ravel(), y_pred_binarize.ravel())
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
    
    return (roc_auc)

### Predict data by each user

In [ ]:
# create users bases on user_id in the database
users = [i for i in range(112,185,1)] # 40 
# users = [i for i in range(203,258,1)]

In [ ]:
# define variables
f1_test_all = []
f1_val_all = []
n_classes_all = []
index = []
i=0
auc_test_all = []
auc_val_all = []
auc_class_all = []
user_all = []

for user in users:
    
    user_id = user
    
    filename_model = "users-models/model_user_"+str(user_id)+".sav"
    filename_objs = "users-models/model_user_"+str(user_id)+".pkl"
    
    # load the model from disk
    mcl = load_model(filename_model)
    
    # load the variables from disk
    (training_features, test_features, training_target, 
     test_target, X_train, X_val, y_train, y_val,
     class_weight,class_names,n_classes) = load_objs(filename_objs)
    
    y_pred_test = mcl.predict(test_features)
    predictions_test = [round(value) for value in y_pred_test]
    
    y_pred_val = mcl.predict(X_val)
    predictions_val = [round(value) for value in y_pred_val]
    
    roc_auc_test = rocauc_perclass(y_pred_test, test_target, class_names, n_classes)
    roc_auc_val = rocauc_perclass(y_pred_val, y_val, class_names, n_classes)
    
    j=0
    for name in class_names:
        user_all.append(user_id)
        auc_test_all.append(roc_auc_test[j])
        auc_val_all.append(roc_auc_val[j])
        n_classes_all.append(n_classes)
        auc_class_all.append(name)
        j+=1
    
    np.set_printoptions(precision=2)
    
d = {
    'user_id':user_all,
    'auc_test': auc_test_all, 
    'auc_val': auc_val_all,
    'auc_class': auc_class_all,
    'n_classes': n_classes_all,
    }
df = pd.DataFrame(data=d)

### Export results

In [ ]:
def print_full(data):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        print(data)

In [ ]:
# print_full(df.sort_values(by=['f1_test'],ascending=False))
df = df.dropna()
df.describe()
df.head()
df = pd.DataFrame(df, columns=['user_id','auc_test','auc_val','auc_class','n_classes'])
df.to_csv('data/accuracy.csv')